In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch

class SemanticSearch:
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2'):
        """
        Инициализация модели для семантического поиска

        Args:
            model_name: название предобученной модели
        """
        self.model = SentenceTransformer(model_name)
        self.corpus = []
        self.corpus_embeddings = None

    def add_documents(self, documents):
        """
        Добавление документов в поисковый индекс

        Args:
            documents: список текстовых документов
        """
        self.corpus = documents
        print("Создание эмбеддингов для документов...")
        self.corpus_embeddings = self.model.encode(
            self.corpus,
            convert_to_tensor=True,
            show_progress_bar=True
        )

    def search(self, query, top_k=5):
        """
        Поиск наиболее релевантных документов

        Args:
            query: поисковый запрос
            top_k: количество возвращаемых результатов

        Returns:
            список кортежей (документ, оценка схожести)
        """
        if self.corpus_embeddings is None:
            raise ValueError("Сначала добавьте документы с помощью add_documents()")

        # Кодируем запрос
        query_embedding = self.model.encode(query, convert_to_tensor=True)

        # Вычисляем косинусное сходство
        cos_scores = util.cos_sim(query_embedding, self.corpus_embeddings)[0]

        # Получаем топ-K результатов
        top_results = torch.topk(cos_scores, k=min(top_k, len(self.corpus)))

        results = []
        for score, idx in zip(top_results.values, top_results.indices):
            results.append({
                'document': self.corpus[idx],
                'score': score.item(),
                'index': idx.item()
            })

        return results

    def batch_search(self, queries, top_k=5):
        """
        Поиск по нескольким запросам одновременно
        """
        query_embeddings = self.model.encode(queries, convert_to_tensor=True)
        cos_scores = util.cos_sim(query_embeddings, self.corpus_embeddings)

        all_results = []
        for i, query_scores in enumerate(cos_scores):
            top_results = torch.topk(query_scores, k=min(top_k, len(self.corpus)))

            query_results = []
            for score, idx in zip(top_results.values, top_results.indices):
                query_results.append({
                    'document': self.corpus[idx],
                    'score': score.item(),
                    'index': idx.item()
                })
            all_results.append(query_results)

        return all_results

# Пример использования
def demo_semantic_search():
    # Создаем коллекцию документов
    documents = [
        "Коты являются домашними животными и любят спать много часов",
        "Собаки верные друзья человека и любят играть с мячом",
        "Программирование на Python требует понимания основных структур данных",
        "Машинное обучение использует алгоритмы для обучения на данных",
        "Нейронные сети являются основой глубокого обучения",
        "Погода в Москве сегодня солнечная и теплая",
        "Банки предлагают различные финансовые услуги для населения",
        "Рестораны в центре города предлагают разнообразную кухню",
        "Путешествия по Европе становятся все более популярными",
        "Здоровый образ жизни включает правильное питание и спорт"
    ]

    # Инициализируем поиск
    search_engine = SemanticSearch()
    search_engine.add_documents(documents)

    # Тестовые запросы
    queries = [
        "животные которые живут дома",
        "искусственный интеллект и данные",
        "еда и питание"
    ]

    for query in queries:
        print(f"\n=== Результаты для запроса: '{query}' ===")
        results = search_engine.search(query, top_k=3)

        for i, result in enumerate(results, 1):
            print(f"{i}. [Сходство: {result['score']:.4f}] {result['document']}")

if __name__ == "__main__":
    demo_semantic_search()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Создание эмбеддингов для документов...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


=== Результаты для запроса: 'животные которые живут дома' ===
1. [Сходство: 0.8091] Коты являются домашними животными и любят спать много часов
2. [Сходство: 0.6127] Машинное обучение использует алгоритмы для обучения на данных
3. [Сходство: 0.6040] Здоровый образ жизни включает правильное питание и спорт

=== Результаты для запроса: 'искусственный интеллект и данные' ===
1. [Сходство: 0.7031] Машинное обучение использует алгоритмы для обучения на данных
2. [Сходство: 0.6081] Коты являются домашними животными и любят спать много часов
3. [Сходство: 0.5905] Банки предлагают различные финансовые услуги для населения

=== Результаты для запроса: 'еда и питание' ===
1. [Сходство: 0.6213] Путешествия по Европе становятся все более популярными
2. [Сходство: 0.6151] Банки предлагают различные финансовые услуги для населения
3. [Сходство: 0.5902] Здоровый образ жизни включает правильное питание и спорт


In [3]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [4]:
#import faiss
import pickle
import os
from pathlib import Path

class AdvancedSemanticSearch:
    def __init__(self, model_name='sentence-transformers/all-MiniLM-L6-v2',
                 index_path="search_index"):
        self.model = SentenceTransformer(model_name)
        self.index_path = Path(index_path)
        self.corpus = []
        self.index = None
        self.embeddings = None

    def create_index(self, documents, index_name="corpus_index"):
        """
        Создание FAISS индекса для быстрого поиска
        """
        self.corpus = documents

        print("Создание эмбеддингов...")
        self.embeddings = self.model.encode(documents, show_progress_bar=True)

        # Нормализуем векторы для косинусного сходства
        faiss.normalize_L2(self.embeddings)

        # Создаем FAISS индекс
        dimension = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dimension)  # Inner Product для косинусного сходства
        self.index.add(self.embeddings.astype('float32'))

        # Сохраняем индекс
        self._save_index(index_name)

    def _save_index(self, index_name):
        """Сохранение индекса и метаданных"""
        self.index_path.mkdir(exist_ok=True)

        # Сохраняем FAISS индекс
        faiss.write_index(self.index, str(self.index_path / f"{index_name}.faiss"))

        # Сохраняем метаданные
        metadata = {
            'corpus': self.corpus,
            'embeddings': self.embeddings
        }

        with open(self.index_path / f"{index_name}_meta.pkl", 'wb') as f:
            pickle.dump(metadata, f)

    def load_index(self, index_name="corpus_index"):
        """Загрузка предварительно созданного индекса"""
        index_file = self.index_path / f"{index_name}.faiss"
        meta_file = self.index_path / f"{index_name}_meta.pkl"

        if not index_file.exists() or not meta_file.exists():
            raise FileNotFoundError("Индекс не найден. Сначала создайте индекс.")

        self.index = faiss.read_index(str(index_file))

        with open(meta_file, 'rb') as f:
            metadata = pickle.load(f)
            self.corpus = metadata['corpus']
            self.embeddings = metadata['embeddings']

    def search(self, query, top_k=5, score_threshold=0.0):
        """
        Поиск с использованием FAISS индекса
        """
        if self.index is None:
            raise ValueError("Сначала создайте или загрузите индекс")

        # Кодируем и нормализуем запрос
        query_embedding = self.model.encode([query])
        faiss.normalize_L2(query_embedding)

        # Поиск в индексе
        scores, indices = self.index.search(query_embedding.astype('float32'), top_k)

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx < len(self.corpus) and score >= score_threshold:
                results.append({
                    'document': self.corpus[idx],
                    'score': float(score),
                    'index': int(idx)
                })

        return results

    def hybrid_search(self, query, top_k=5, keyword_weight=0.3, semantic_weight=0.7):
        """
        Гибридный поиск: семантический + ключевые слова
        """
        # Семантический поиск
        semantic_results = self.search(query, top_k=top_k*2)

        # Простой поиск по ключевым словам
        keyword_results = self._keyword_search(query, top_k=top_k*2)

        # Объединяем и ранжируем результаты
        combined = self._combine_results(semantic_results, keyword_results,
                                       semantic_weight, keyword_weight)

        return combined[:top_k]

    def _keyword_search(self, query, top_k=5):
        """Простой поиск по ключевым словам"""
        query_words = set(query.lower().split())
        results = []

        for idx, doc in enumerate(self.corpus):
            doc_words = set(doc.lower().split())
            common_words = query_words.intersection(doc_words)
            score = len(common_words) / len(query_words) if query_words else 0

            if score > 0:
                results.append({
                    'document': doc,
                    'score': score,
                    'index': idx,
                    'type': 'keyword'
                })

        return sorted(results, key=lambda x: x['score'], reverse=True)[:top_k]

    def _combine_results(self, semantic_results, keyword_results,
                        semantic_weight, keyword_weight):
        """Объединение результатов семантического и ключевого поиска"""
        combined = {}

        # Добавляем семантические результаты
        for result in semantic_results:
            idx = result['index']
            combined[idx] = {
                'document': result['document'],
                'score': result['score'] * semantic_weight,
                'index': idx,
                'semantic_score': result['score'],
                'keyword_score': 0
            }

        # Добавляем/обновляем ключевые результаты
        for result in keyword_results:
            idx = result['index']
            if idx in combined:
                combined[idx]['score'] += result['score'] * keyword_weight
                combined[idx]['keyword_score'] = result['score']
            else:
                combined[idx] = {
                    'document': result['document'],
                    'score': result['score'] * keyword_weight,
                    'index': idx,
                    'semantic_score': 0,
                    'keyword_score': result['score']
                }

        return sorted(combined.values(), key=lambda x: x['score'], reverse=True)

# Демонстрация работы
def demo_advanced_search():
    # Загрузка датасета (пример)
    from sklearn.datasets import fetch_20newsgroups

    print("Загрузка данных...")
    newsgroups = fetch_20newsgroups(subset='train',
                                  categories=['sci.space', 'comp.graphics'],
                                  remove=('headers', 'footers', 'quotes'))
    documents = newsgroups.data[:100]  # Берем первые 100 документов

    # Инициализация и создание индекса
    search_engine = AdvancedSemanticSearch()

    if not (search_engine.index_path / "corpus_index.faiss").exists():
        print("Создание поискового индекса...")
        search_engine.create_index(documents)
    else:
        print("Загрузка существующего индекса...")
        search_engine.load_index()

    # Тестирование поиска
    test_queries = [
        "космос и планеты",
        "компьютерная графика",
        "научные исследования",
        "технологии и программирование"
    ]

    for query in test_queries:
        print(f"\n" + "="*60)
        print(f"ЗАПРОС: '{query}'")
        print("="*60)

        # Семантический поиск
        print("\n--- Семантический поиск ---")
        results = search_engine.search(query, top_k=3)
        for i, result in enumerate(results, 1):
            print(f"{i}. [Score: {result['score']:.4f}] {result['document'][:100]}...")

        # Гибридный поиск
        print("\n--- Гибридный поиск ---")
        hybrid_results = search_engine.hybrid_search(query, top_k=3)
        for i, result in enumerate(hybrid_results, 1):
            print(f"{i}. [Total: {result['score']:.4f} | Semantic: {result['semantic_score']:.4f} | Keyword: {result['keyword_score']:.4f}]")
            print(f"   {result['document'][:100]}...")

if __name__ == "__main__":
    demo_advanced_search()

Загрузка данных...


/usr/local/lib/python3.12/dist-packages/sklearn/datasets/_base.py:1519: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5975967
  warnings.warn(f"Retry downloading from url: {remote.url}")


HTTPError: HTTP Error 403: Forbidden